The intuition behind gradient boosting is that we start off with any random model, usually being an unoptimized tree, and then, based off its predictions, optimize the bias. Since most models rely on $\hat{y}+bias$ to get the real prediction, we're pretty much just approximating our function, but through the bias term rather than coefficients.

#### Intuitive description of the algorithm
- The starting model, based on which we are going to optimize the bias, could be anything, even the mean value across all predictions. By plugging $X$ we get initial predictions $\hat{y}$;
- We calculate residuals $e=y-\hat{y}$. Now we build a "weak learner" for the bias term. For example, it could be a decision tree that predicts the bias. Not that in the regression problem if there are more than one value left in the leaf node, we take the average to yeild the prediction. For more generalization we plug $\eta$ as a learning rate parameter to the predicted residuals;
- We calculate new predictions based on $\hat{y}+\eta\times e_1$ and then repeat the process until we reach $M$ iterations or a desired quality

#### Mathematically strictier description
- Initially we want to find such an arbitrary function $\hat{f}(x)$ that minimizes $y\approx\hat{f}(x)$. That means that $\hat{f}(x)=argmin_{f(x)}L(y,f(x))$, where $L(y,f)$ is some **differentiable** loss function
- Since the range of possible functions approximating $f(x)$ is infinite, we limit the problem to a family of functions $f(x,\theta), \theta\in R^d$ (for example a decision tree) and transform the problem to finding best $\theta$ instead of some "function": $\hat{\theta}=argmin_{\theta}E_{x,y}[L(y,f(x,\theta))]$, where $E$ - expected value or mean. 
    > **Improtant reminder** is that just like in every other NN we get best parameters by applying gradient descend with some multiplication by $\eta\in(0,1]$, i.e. each parameter actually consists of a negative sum of losses with preadded initial value of the parameter. Therefore, for better intuition we can rewrite the optimization problem as $\hat{\theta}=\sum^N_{i=1}L(y_i,f(x_i,\hat{\theta}))$
- Right now it seems that GBMs are nothing more but a fancier way to describe regular linear models. To elevate confusion it is important to say that GBMs approximate a function as a sum of incremental improvements, **each being a function**, i.e. $\hat{f}(x)=\sum_{i=0}^M\hat{f_i}(x)$, where $\hat{f}(x)$ is limited by some function family $\hat{f}(x)=h(x,\theta)$. Moreover on every step of finding another function (from now on we will also refer to them as "models") we also need to select an optimal $\rho\in R$.


In [1]:
import numpy as np
from sklearn.metrics import accuracy_score

class Loss(object):
    def loss(self, y_true, y_pred):
        return NotImplementedError()

    def gradient(self, y, y_pred):
        raise NotImplementedError()

    def acc(self, y, y_pred):
        return 0

class SquareLoss(Loss):
    def __init__(self): pass

    def loss(self, y, y_pred):
        return 0.5 * np.power((y - y_pred), 2)

    def gradient(self, y, y_pred):
        return -(y - y_pred)

class CrossEntropy(Loss):
    def __init__(self): pass

    def loss(self, y, p):
        # Avoid division by zero
        p = np.clip(p, 1e-15, 1 - 1e-15)
        return - y * np.log(p) - (1 - y) * np.log(1 - p)

    def acc(self, y, p):
        return accuracy_score(np.argmax(y, axis=1), np.argmax(p, axis=1))

    def gradient(self, y, p):
        # Avoid division by zero
        p = np.clip(p, 1e-15, 1 - 1e-15)
        return - (y / p) + (1 - y) / (1 - p)

In [2]:
from sklearn.tree import DecisionTreeRegressor
from tqdm import tqdm

class gbm():
    
    def  __init__(self, eta, m, min_samples, min_impurity, 
                  max_depth, is_reg) -> None:
        """ 
        eta: learning rate
        m: number of weak learners
        """
        self.eta = eta
        self.m = m
        self.min_samples = min_samples
        self.min_impurity = min_impurity
        self.max_depth = max_depth
        self.is_reg = is_reg

        self.loss = SquareLoss()
        if not self.is_reg:
            self.loss = CrossEntropy()

        # initialize estimators, e.g. trees
        self.tree_estimators = []
        for _ in range(self.m):
            self.tree_estimators.append(
                DecisionTreeRegressor(
                    min_samples_split=self.min_samples,
                    min_impurity_decrease=self.min_impurity,
                    max_depth=self.max_depth)
            )
        
    def _softmax(self, values):
        return np.exp(values) /\
            np.expand_dims(np.sum(np.exp(values, axis=1)),axis=1)

    def fit(self, X, y):
        y_pred = np.full(y.shape,y.mean())
        for i in tqdm(range(self.m)):
            grad = self.loss.gradient(y, y_pred)
            self.tree_estimators[i].fit(X, grad)
            weak_learner = self.tree_estimators[i].predict(X)
            y_pred -= self.eta * weak_learner

    def predict(self, X):
        y_pred = np.array([])

        # run the sample through ensemble of weak learners
        # recursively from bottom to the initial prediction
        for tree in self.tree_estimators: 
            weak_learner = self.eta * tree.predict(X)
            y_pred = -weak_learner if \
                not y_pred.any() else y_pred - weak_learner
        
        if not self.is_reg:
            # apply softmax to predicted estimated values
            # to transform values into [0,1] "probability" dist
            y_pred = np.argmax(self._softmax(y_pred),axis=1)

        return y_pred

GBM is a universal approach to both regression and classifcation problems, therefore we need to build additional child classes for each type of the problem:

In [10]:
class gbmRegressor(gbm):
    def __init__(self, n_estimators=200, eta=0.5, min_samples_split=2,
                 min_var_red=1e-7, max_depth=4):
        super().__init__(m=n_estimators, 
            eta=eta, 
            min_samples=min_samples_split, 
            min_impurity=min_var_red,
            max_depth=max_depth,
            is_reg=True)

class gbmClassifier(gbm):
    def __init__(self, n_estimators=200, eta=.5, min_samples_split=2,
                 min_info_gain=1e-7, max_depth=2):
        super().__init__(m=n_estimators, 
            eta=eta, 
            min_samples=min_samples_split, 
            min_impurity=min_info_gain,
            max_depth=max_depth,
            is_reg=False)

Now we can compare, for instance, our GBM for regression with Scikit-learn implementation on some arbitrary regression dataset. We can see that $R^2=1-\frac{\text{Sum of squares of residuals from regression}}{\text{Sum of squares of residuals against the average}}$, which explaines how much space variance in regression takes up in the explained variance in the dataset is slightly lower than in Scikit-learn, which means that a $MSE_{friedman}$ used as a criterion for the split quality in Scikit-learn's GBM implementation is better than the regular squared error used in the Tree regressor by default:

In [4]:
from sklearn.datasets import make_regression
from sklearn.model_selection import train_test_split
X, y = make_regression(random_state=0)
X_train, X_test, y_train, y_test = train_test_split(
    X, y, random_state=0)

In [14]:
# our own implementation
from sklearn.metrics import r2_score, mean_squared_error
customReg = gbmRegressor(eta=0.1, n_estimators=100,
                         min_samples_split=2)
customReg.fit(X_train, y_train)
predictions = customReg.predict(X_test)
print(f'R^2: {r2_score(y_test, predictions)} :: ' \
      f'MSE: {mean_squared_error(y_test, predictions)}')

100%|██████████| 100/100 [00:00<00:00, 184.76it/s]

R^2: 0.3920367384803918 :: MSE: 11929.734155842236


In [15]:
# sklearn implementation
from sklearn.ensemble import GradientBoostingRegressor
reg = GradientBoostingRegressor(random_state=0)
reg.fit(X_train, y_train)
predictions_sklearn = reg.predict(X_test)
print(f'R^2: {r2_score(y_test, predictions_sklearn)} :: ' \
      f'MSE: {mean_squared_error(y_test, predictions_sklearn)}')

R^2: 0.43542564320925925 :: MSE: 11078.337152946428
